# OpenAI API: Code Interpreter and Advanced Data Analysis
## 03_03 - Troubleshoot using logs

### Install the necessary libraries. 

In [ ]:
pip install openai

In [ ]:
pip install openai[datalib]

In [ ]:
pip install urllib3==1.26.6 

In [ ]:
pip install python-dotenv

### Import the libraries and environment file to gain access to the Open API Key
#### The key can be generated here: https://platform.openai.com/account/api-keys

In [241]:
#import OS, OpenAI, and time modules
import openai
import os
from openai import OpenAI
import time


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

### Authenticate to the API using the API Key
#### Pull from environment variables or use api_key = ("your_key_here") to hardcode the key

In [242]:
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']  
)

### Helper function to upload Python example files

In [243]:
def upload_file(path):
    try:
        file = client.files.create(
            file=open(path, "rb"),
            purpose='assistants'
        )
        
        return file.id
    except Exception as e:
        print(e)
        return e

### Helper function to setup an Assistant
#### The Assistant is a Coding Bot
Code Interpreter allows the Assistants API to write and run Python code in a sandboxed execution environment.

In [244]:
def create_assistant(file_id):
    try:
        assistant = client.beta.assistants.create(
            name = "Coding Bot",
            instructions='''Generate a file, always. You are an expert travel
                            agent. When given a list of cities, you plot those cities
                            on a map and make the map image available for download. If 
                            you write Python code, also make the file available for 
                            download.''',
            model="gpt-4-1106-preview",
            tools=[{"type": "code_interpreter"}],
            file_ids=[file_id]
        )
        
        return assistant.id
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error 

### Helper function to create a thread to query an Assistant

In [245]:
def query_assistant(assistant_id, user_query):
    try:
        thread = client.beta.threads.create(
            messages=[
                {
                "role": "user",
                "content": user_query
                }
            ]
        )
        
        return thread.id
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

### Helper function to create a run object to execute the thread

In [246]:
def run(assistant_id, thread_id):
    try:
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id
        )
        
        time.sleep(10)
        
        while True:
            print(f'{run.id=} {run.status=}')

            run = client.beta.threads.runs.retrieve(
                thread_id=thread_id,
                run_id=run.id
            )

            status = run.status

            if status == 'completed':
                break
            else:
                time.sleep(10) 

        print(f'{run.id=} {run.status=}')
        
        #retrieve the messages 
        messages = client.beta.threads.messages.list(
            thread_id=thread_id
        )

        #loop and print the messages out
        for thread_message in messages.data:
            # Accessing the content array within each ThreadMessage
            for content in thread_message.content:
                # Checking if the content type is MessageContentText
                if content.type == 'text':
                    # Accessing the text attribute of the MessageContentText
                    text_content = content.text.value
                    print(text_content)
                
        download_file(messages)
        
        return run.id
        
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

### Helper function to download the map

In [247]:
def download_file(messages):     
    try:
        for thread_message in messages.data:
            # Accessing the content array within each ThreadMessage
            for content in thread_message.content:
                # Checking if the content type is 'text'
                if content.type == 'text':
                    # Accessing the annotations within the text content
                    for annotation in content.text.annotations:
                        # Checking for file_path type in annotations
                        if annotation.type == 'file_path':
                            # Extracting the file_id
                            file_id = annotation.file_path.file_id
                            # Extracting the file path
                            file_path = annotation.text

                            # Check if the file path contains '.png'
                            if '.png' in file_path:
                                image_file_id = file_id
                            elif '.py' in file_path:
                                code_file_id = file_id
                                
        
        #get file name
        file_name = client.files.with_raw_response.retrieve_content(image_file_id) 

        #download and save the file locally
        with open("./travel_map.png", "wb") as file:  
            file.write(file_name.content) 
            
        #clean up file
        client.files.delete(image_file_id)
        
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error
    except Exception as e:
        print(e)
        return e

### Helper function to check the logs

In [258]:
# Function to retrieve the steps of a specific run
def get_code_interpreter_logs(thread_id, run_id):
    try:
        # List the steps for the specified thread and run
        steps = client.beta.threads.runs.steps.list(
          thread_id=thread_id,
          run_id=run_id
        )
        
        #loop and print the messages out
        for step in steps:
            # Check if the step is a tool call and involves the code interpreter
            if step.type == 'tool_calls':
                for tool_call in step.step_details.tool_calls:
                    if tool_call.type == 'code_interpreter':
                        # Print the code input
                        print("\n------Code Input:------\n", tool_call.code_interpreter.input)
                         # Print the code outputs (logs)
                        for output in tool_call.code_interpreter.outputs:
                            if output.type == 'image':
                                print("\n------Code Output Logs:------\n", output)
                            else:
                                print("\n------Code Output Logs:------\n", output.logs)
                        print("==================================\n")    

    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

### Read in the Python example file

In [249]:
file_id = upload_file('visits.xlsx')

print(file_id)

file-Z9vPL0ZogBuOGpj5PXNfEDfw


### Create a run object to execute the thread

In [250]:
assistant_id = create_assistant(file_id)

In [251]:
print(assistant_id)

asst_xbIxqqTsQYCwGfvHjPt0KNbe


In [252]:
user_query = '''Plot the given list of cities on the image of a world map using red pins. 
                The plot can be estimates and doesn't need to be 100% accurate. 
                Provide the file id of the image available for download.'''

thread_id = query_assistant(assistant_id, user_query)
print(thread_id)

thread_kF4eg946YgQ43cPXy43cnCJZ


In [253]:
run_id = run(assistant_id, thread_id)

run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='queued'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progress'
run.id='run_CC9lUGgFUDET3xzGZ0jXkwvH' run.status='in_progres

### Display image inline

In [254]:

from IPython.display import Image

image = "./travel_map.png"
print(image)

Image(url=image)

./travel_map.png


### Review the logs

In [259]:
get_code_interpreter_logs(thread_id, run_id)


------Code Input:------
 import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import random

# Load a simple world map
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Create a GeoDataFrame for the cities (using an approximation)
# Since we do not have coordinates, we will place each city in the geometrical center of its country
# Note: This is a rough estimate just for the purpose of the assignment
def get_approx_location(country):
    """Approximate the location of a city by the country's geometrical center."""
    if country in world['name'].values:
        # Get country's geometry
        country_geom = world.loc[world['name'] == country, 'geometry'].values[0]
        # Get the geometrical center (centroid) of the country
        center = country_geom.centroid
        # Introduce some randomness to separate cities in the same country a bit
        random_offset = Point(center.x + random.uniform(-0.5, 0.5), center.y + rand

### Clean up Assistants
Do not leave Assistants or files running for longer periods

In [260]:
client.beta.assistants.delete(assistant_id)

AssistantDeleted(id='asst_xbIxqqTsQYCwGfvHjPt0KNbe', deleted=True, object='assistant.deleted')

In [261]:
client.beta.threads.delete(thread_id)

ThreadDeleted(id='thread_kF4eg946YgQ43cPXy43cnCJZ', deleted=True, object='thread.deleted')